# Aprendizaje de Máquina con Big Data y Apache Spark
## Entrenamiento de modelos y evaluación
![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

Para el procesamiento de Big Data utilizando Ciencia de Datos/Analítica de Datos/Aprendizaje de Máquina, existe una metodología ampliamente utilizada en la industria conocida como CRISP-DM creada por IBM, el siguiente diagrama representa la secuencia de pasos correspondientes a esta metodología:

![CRISP-DM](https://www.ibm.com/docs/es/SS3RA7_sub/modeler_crispdm_ddita/clementine/images/crisp_process.jpg)

En este diagrama se identifican las siguientes etapas:

1. Entendimiento del negocio (objetivos)
2. Exploración de los datos
3. Preparación de los datos
4. Modelado
5. Evaluación
6. Despliegue

El cual representa un proceso iterativo, que comienza con el entendimiento del negocio y termina, y vuelve a comenzar, con la evaluación de resultados.

Este notebook está inspirado en el capítulo 2 del libro [Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb)

## Descripción del problema

Usted es el científico de datos de una empresa de bienes raíces en el estado de California de los Estados Unidos de América. Desde hace un tiempo, se ha identificado una gran dificultad a la hora de asignar un precio acertado a una propiedad para ponerla en el mercado, aumentando en gran medida el tiempo que le toma a un agente concretar la venta de la propiedad. En la mayoría de los casos, se ha evidenciado que el precio inicial está por encima del precio real de mercado, lo que hace la propiedad poco atractiva para los compradores; aunque tampoco se descarta que para algunas propiedades se haya listado un precio menor al del mercado, reduciendo el beneficio económico de la compañia y los agentes de venta.

En este panorama, se le ha asignado la tarea de predecir el valor promedio de las propiedades en el estado de California a partir de un [conjunto de datos](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) con características relevantes.

### Configuración del ambiente de Google Colaboratory

In [ ]:
# Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Next, we will install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# Now, we just need to unzip that folder.
!tar xf spark-3.3.2-bin-hadoop3.tgz

# Setting JVM and Spark path variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# Installing required packages
!pip install pyspark==3.3.2
!pip install findspark

In [2]:
import os
import tarfile
import urllib.request
from pathlib import Path

def fetch_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")

fetch_housing_data()

In [3]:
import datetime as dt
import findspark
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyspark.ml as ml
from pyspark.sql import functions as fct
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType

findspark.init()

### Crear Sesión de Spark e importar los datos

In [4]:
ss = (SparkSession
      .builder
      .appName("data_exploration_preparation")
      .getOrCreate())

In [5]:
path = "/content/datasets/housing/housing.csv"
housing_data = ss.read.csv(path, inferSchema=True, header=True)

### División en el conjunto de entrenamiento y conjunto de evaluación

In [6]:
train_size = 0.7 # Tamaño del conjunto de entrenamiento: 70%
test_size = 0.3 # Tamaño del conjunto de evaluación: 30%
housing_data_train, housing_data_test = housing_data.randomSplit([train_size, test_size], seed=42)
housing_data_pd = housing_data_train.toPandas() # Convertirlo a un DataFrame de pandas para generar visualizaciones

### Creación del Pipeline de preprocesamiento de los datos

<!--

sqlTrans = ml.feature.SQLTransformer(statement="SELECT scaled_longitude AS longitude, \
                                                       scaled_latitude AS latitude, \
                                                       scaled_housing_median_age AS housing_median_age, \
                                                       scaled_total_rooms AS total_rooms, \
                                                       scaled_total_bedrooms_complete AS total_bedrooms, \
                                                       scaled_population AS population, \
                                                       scaled_households AS households, \
                                                       scaled_median_income AS median_income, \
                                                       onehot_ocean_proximity AS ocean_proximity, \
                                                       features, \
                                                       median_house_value AS label FROM __THIS__")


-->

In [7]:
imputer = ml.feature.Imputer(strategy="median",inputCols=["total_bedrooms"],outputCols=["total_bedrooms_complete"])

stringIndexer = ml.feature.StringIndexer(inputCol="ocean_proximity", outputCol="ordinal_ocean_proximity", stringOrderType="frequencyDesc")

onehotencoder = ml.feature.OneHotEncoder(inputCol="ordinal_ocean_proximity", outputCol="onehot_ocean_proximity")

columns_to_scale = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms_complete", "population", "households", "median_income"]

assemblers = [ml.feature.VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]#+["median_house_value"]]

scalers = [ml.feature.MinMaxScaler(inputCol=col + "_vec", outputCol="scaled_" + col) for col in columns_to_scale]

feature_assembler = ml.feature.VectorAssembler(inputCols=["scaled_" + col for col in columns_to_scale]+["onehot_ocean_proximity"], outputCol="features")

sqlTrans = ml.feature.SQLTransformer(statement="SELECT features, median_house_value AS label FROM __THIS__")

preprocess_pipeline = ml.Pipeline(stages=[imputer, stringIndexer, onehotencoder]+assemblers+scalers+[feature_assembler, sqlTrans])

In [ ]:
pipeline_model = preprocess_pipeline.fit(housing_data_train)
pipeline_model.transform(housing_data_train).show()

### Entrenamiento y evaluación de modelos

*Entrenar* un modelo se refiere a configurar sus parámetros de forma que se ajusten de la mejor manera a los datos del conjunto de entrenamiento. Para esto se hace necesario definir una medida que nos indique qué tan bien (o mal) se está realizando ese ajuste de los parámetros, y al final que conjunto de parámetros obtiene el mejor resultado; nos referimos a esta medida como **métrica** o **función de desempeño**. Para tareas de regresión, la métrica más utilizada es la raíz cuadrada del error cuadrático medio o RMSE (*Root Mean Squared Error*), definido como:

$$RMSE(X,\theta)=\sqrt{\frac{1}{m}\sum_{i=1}^m(\theta^Tx^{(i)}-y^{(i)})^2}$$

Donde $X$ representa nuestro conjunto de entrenamiento con $m$ muestras, $x^{(i)}$ representa los valores de las características de entrada para una muestra y $y^{(i)}$ la etiqueta o valor objetivo; y $\theta$ representa los parámetros del modelo. $\theta^Tx^{(i)}$ corresponde a las predicciones del modelo, por lo que también lo podríamos expresar como $\hat y$.

#### Modelo de regresión lineal

La regresión lineal consiste en ajustar una línea recta o un plano a los datos para realizar predicciones de un valor númerico. Para una variable, este modelo se puede expresar matemáticamente como:

$$\hat y=\theta_0+\theta_1x$$

Lo cual podemos interpretar como un modelo donde la predicción es una suma ponderada de las variables de entrada.

Generalizando para $n$ variables:

$$\hat y=\theta_0+\theta_1x_1+\theta_2x_2+...+\theta_nx_n$$

Donde:
- $\hat y$ es la predicción
- $n$ es el número de variables o características
- $x_i$ es el valor de la característica $i$-ésima
- $\theta_j$ es el $j$-ésimo parámetro del modelo, incluyendo el intercepto $\theta_0$ y los pesos de las características

In [ ]:
linear_regression = ml.regression.LinearRegression(maxIter=1000)
model_pipeline = ml.Pipeline(stages=[preprocess_pipeline, linear_regression])

model = model_pipeline.fit(housing_data_train)

model.transform(housing_data_train).show()

In [ ]:
# Obtener el modelo a partir del pipeline
lrModel = model.stages[-1]

# Imprimir los pesos y el intercepto para el modelo de regresión lineal
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Resumen del modelo y algunas métricas sobre los datos de entrenamiento
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

In [ ]:
predictions = model.transform(housing_data_test)
predictions.show(5)
evaluator = ml.evaluation.RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

#### Modelo de árboles de decisión

Un árbol de decisión es un algoritmo de aprendizaje supervisado no paramétrico, que se utiliza tanto para tareas de clasificación como de regresión. Tiene una estructura de árbol jerárquica, que consta de un nodo raíz, ramas, nodos internos y nodos hoja [[1]](https://www.ibm.com/es-es/topics/decision-trees).

![Árbol de decisión](https://www.ibm.com/content/dam/connectedassets-adobe-cms/worldwide-content/cdp/cf/ul/g/df/de/Decision-Tree.component.xl.ts=1666031329586.png/content/adobe-cms/es/es/topics/decision-trees/jcr:content/root/table_of_contents/intro/complex_narrative/items/content_group_1423241468/image)

In [ ]:
decisionTree = ml.regression.DecisionTreeRegressor()
model_pipeline = ml.Pipeline(stages=[preprocess_pipeline, decisionTree])

model = model_pipeline.fit(housing_data_train)

train_predictions = model.transform(housing_data_train)
train_predictions.show()

In [ ]:
print(model.stages[-1].featureImportances)

In [ ]:
evaluator = ml.evaluation.RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(train_predictions)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse)

In [ ]:
predictions = model.transform(housing_data_test)
predictions.show(5)
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

#### Modelos de ensamble: Random Forest

El random forest es un algoritmo de machine learning de uso común registrado por Leo Breiman y Adele Cutler, que combina la salida de múltiples árboles de decisión para alcanzar un solo resultado. Su facilidad de uso y flexibilidad han impulsado su adopción, ya que maneja problemas de clasificación y regresión. 

El algoritmo de random forest se compone de un conjunto de árboles de decisión, y cada árbol del conjunto se compone de una muestra de datos extraída de un conjunto de entrenamiento con reemplazo. Utiliza la aleatoriedad de características para crear un bosque no correlacionado de árboles de decisión [[2]](https://www.ibm.com/es-es/topics/random-forest)

In [ ]:
randomForest = ml.regression.RandomForestRegressor()
model_pipeline = ml.Pipeline(stages=[preprocess_pipeline, randomForest])

model = model_pipeline.fit(housing_data_train)

train_predictions = model.transform(housing_data_train)
train_predictions.show()

In [ ]:
print(model.stages[-1].featureImportances)

In [ ]:
evaluator = ml.evaluation.RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(train_predictions)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse)

In [ ]:
predictions = model.transform(housing_data_test)
predictions.show(5)
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

### Ajuste de los modelos

La siguiente celda tarda en ejecutar alrededor de 16 minutos.

In [11]:
paramGrid = (ml.tuning.ParamGridBuilder()
             .addGrid(imputer.strategy, ["mean", "mode"])
             .addGrid(randomForest.numTrees, [20, 50, 100])
             .addGrid(randomForest.maxDepth, [5, 10, 15])
             .build())

crossval = ml.tuning.CrossValidator(estimator=model_pipeline,
                                    estimatorParamMaps=paramGrid,
                                    evaluator=ml.evaluation.RegressionEvaluator(metricName="rmse"),
                                    numFolds=2)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(housing_data_train)

In [17]:
cvModel.avgMetrics

[69021.84464164326,
 57870.5792590263,
 55793.25891993652,
 68346.14602693476,
 56972.30227252983,
 54477.309875511855,
 68615.760966357,
 57011.33817286771,
 54332.94862000374,
 69007.19441872317,
 57790.46496921142,
 55603.95241932221,
 68395.882071058,
 56930.328801938944,
 54455.43365359685,
 68518.87799900028,
 56833.62248913152,
 54177.17681773282]

In [18]:
cvModel.stdMetrics

[347.8015940742116,
 405.41841137834126,
 715.1965414721162,
 443.35799159102316,
 770.5886910378322,
 716.4265209351761,
 369.6458446341421,
 429.1892733001878,
 543.1629224696808,
 354.79779027934273,
 221.39009861246086,
 506.86581369236956,
 501.61918342389254,
 502.74413407020256,
 596.8556373187093,
 265.49832042490743,
 297.4278743308023,
 528.8094796218829]

In [16]:
cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)]

{Param(parent='Imputer_3f27637fec74', name='strategy', doc='strategy for imputation. If mean, then replace missing values using the mean value of the feature. If median, then replace missing values using the median value of the feature. If mode, then replace missing using the most frequent value of the feature.'): 'mean',
 Param(parent='RandomForestRegressor_5a46af05f3fd', name='numTrees', doc='Number of trees to train (>= 1).'): 20,
 Param(parent='RandomForestRegressor_5a46af05f3fd', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5}

In [22]:
ModelsParams = []
for modelsParams in cvModel.getEstimatorParamMaps():
  params = []
  for key, item in modelsParams.items():
    params.append(f"{key.name}: {item}")
  ModelsParams.append(" - ".join(params))

In [24]:
for params, mean, std in zip(ModelsParams, cvModel.avgMetrics, cvModel.stdMetrics):
  print(f"Para los parámetros {params}: \n Se obtuvo un rmse de {mean} ± {std} \n\n")

Para los parámetros strategy: mean - numTrees: 20 - maxDepth: 5: 
 Se obtuvo un rmse de 69021.84464164326 ± 347.8015940742116 


Para los parámetros strategy: mean - numTrees: 20 - maxDepth: 10: 
 Se obtuvo un rmse de 57870.5792590263 ± 405.41841137834126 


Para los parámetros strategy: mean - numTrees: 20 - maxDepth: 15: 
 Se obtuvo un rmse de 55793.25891993652 ± 715.1965414721162 


Para los parámetros strategy: mean - numTrees: 50 - maxDepth: 5: 
 Se obtuvo un rmse de 68346.14602693476 ± 443.35799159102316 


Para los parámetros strategy: mean - numTrees: 50 - maxDepth: 10: 
 Se obtuvo un rmse de 56972.30227252983 ± 770.5886910378322 


Para los parámetros strategy: mean - numTrees: 50 - maxDepth: 15: 
 Se obtuvo un rmse de 54477.309875511855 ± 716.4265209351761 


Para los parámetros strategy: mean - numTrees: 100 - maxDepth: 5: 
 Se obtuvo un rmse de 68615.760966357 ± 369.6458446341421 


Para los parámetros strategy: mean - numTrees: 100 - maxDepth: 10: 
 Se obtuvo un rmse de 57

In [14]:
prediction = cvModel.transform(housing_data_test)
prediction.show(5)
rmse = evaluator.evaluate(prediction)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+--------------------+--------+------------------+
|            features|   label|        prediction|
+--------------------+--------+------------------+
|[0.00498007968127...|103600.0|109501.29450353034|
|[0.01195219123505...|106700.0|116568.15092823739|
|[0.01195219123505...| 73200.0|105368.09296504149|
|[0.01294820717131...| 78300.0|  82654.6257902175|
|[0.01593625498007...| 90100.0|162644.31522776833|
+--------------------+--------+------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 51640.5


### Despliegue

La etapa de despliegue se refiere a poner nuestro modelo en producción, sea creando un script que recupere los datos de nuestro sistema de almacenamiento, o mediante la creación de una API (_Application Programming Interface_) que permita a otras aplicaciones o una página web enviar datos para realizar predicciones. Ejemplo de creación de una API para un modelo de ML usando Python y Flask: [https://www.geeksforgeeks.org/](https://www.geeksforgeeks.org/deploy-machine-learning-model-using-flask/)